In [1]:
# FBA problem to for DAHP maximum yield when glucose
# is taken up via phosphotransferase

# This problem is based on the following article:

# Patnaik R and Liao JC, Engineering of Escherichia coli central metabolism for
# aromatic metabolite production with near theoretical yield. Applied and
# Environmental Microbiology 60: 3903-3908, 1994,
# doi: 10.1128/aem.60.11.3903-3908.1994

import cobra
import re
import pandas as pd

reactions = {'in_glc': ' -> glc',
             'transfer_glc': 'glc + pep -> g6p + pyr',
             'glycolysis_1': 'g6p <-> f6p',
             'glycolysis_2': 'f6p -> 2 gap',
             'glycolysis_3': 'gap <-> pep',
             'glycolysis_4': 'pep -> pyr',
             'ppp1': 'g6p -> p5p + co2',
             'ppp2': '2 p5p <-> s7p + gap',
             'ppp3': 's7p + gap <-> f6p + e4p',
             'ppp4': 'p5p + e4p <-> f6p + gap',
             'dahp_syn': 'e4p + pep -> dahp',
             'out_dahp': 'dahp -> ',
             'out_pyr': 'pyr -> ',
             'out_co2': 'co2 -> ',
            }

In [2]:
mets = set()
for eqn in reactions.values():
    for token in re.findall(r'\b[a-zA-Z]\w*\b', eqn):
        mets.add(token)
mets

{'co2', 'dahp', 'e4p', 'f6p', 'g6p', 'gap', 'glc', 'p5p', 'pep', 'pyr', 's7p'}

In [3]:
def parse_side(side):
    stoich = {}
    if not side.strip():
        return stoich

    terms = side.split('+')
    for term in terms:
        term = term.strip()
        m = re.match(r'(?:(\d+)\s+)?(\w+)', term)
        coeff = int(m.group(1)) if m.group(1) else 1
        met = m.group(2)
        stoich[met] = coeff
    return stoich

In [4]:
model = cobra.Model('dahp_synthesis')

met_objs = {
    m: cobra.Metabolite(m, compartment='c')
    for m in mets
}

model.add_metabolites(met_objs.values())

for rid, eqn in reactions.items():
    rxn = cobra.Reaction(rid)

    if '<->' in eqn:
        left, right = eqn.split('<->')
        rxn.lower_bound = -1000
        rxn.upper_bound = 1000
    else:
        left, right = eqn.split('->')
        rxn.lower_bound = 0
        rxn.upper_bound = 1000

    lhs = parse_side(left)
    rhs = parse_side(right)

    stoich = {}
    for m, c in lhs.items():
        stoich[met_objs[m]] = -c
    for m, c in rhs.items():
        stoich[met_objs[m]] = c

    rxn.add_metabolites(stoich)
    model.add_reactions([rxn])

model.reactions.get_by_id('in_glc').bounds = (7, 7)

In [5]:
model.objective = 'dahp_syn'

print(model.summary())
cobra.flux_analysis.find_blocked_reactions(model)

Objective
1.0 dahp_syn = 3.0

Uptake
------
Metabolite Reaction Flux  C-Number C-Flux
       glc   in_glc    7         0  0.00%

Secretion
---------
Metabolite Reaction Flux  C-Number C-Flux
      dahp out_dahp   -3         0  0.00%
       pyr  out_pyr   -7         0  0.00%



[]

In [6]:
sol = model.optimize()

sol.status
sol.objective_value  # optimal objective value

fluxes = sol.fluxes

In [7]:
df = pd.DataFrame({
    'stoichiometry': reactions,
    'flux': fluxes
})

print(df)

                        stoichiometry  flux
in_glc                         -> glc   7.0
transfer_glc   glc + pep -> g6p + pyr   7.0
glycolysis_1              g6p <-> f6p   7.0
glycolysis_2             f6p -> 2 gap   6.0
glycolysis_3              gap <-> pep  10.0
glycolysis_4               pep -> pyr   0.0
ppp1                 g6p -> p5p + co2   0.0
ppp2              2 p5p <-> s7p + gap   1.0
ppp3          s7p + gap <-> f6p + e4p   1.0
ppp4          p5p + e4p <-> f6p + gap  -2.0
dahp_syn            e4p + pep -> dahp   3.0
out_dahp                     dahp ->    3.0
out_pyr                       pyr ->    7.0
out_co2                       co2 ->    0.0
